# Q1: Validate/Coerce Untrusted JSON from LLM

You receive untrusted JSON from an LLM. Validate/coerce it to the schema below.

## Target Schema

```json
{
  "action": "search" | "answer",
  "q": non-empty str,  // required iff action == "search"
  "k": int in [1, 5]  // optional (default: 3)
}
```

## Implementation Requirements

Implement the following method:

```python
from typing import Any, Dict, List, Tuple

def validate_tool_call(payload: Dict[str, Any]) -> Tuple[Dict[str, Any], List[str]]:
    """
    Returns (clean, errors). 'clean' strictly follows the schema with defaults applied.

    Rules:
    - Trim strings; coerce numeric strings to ints.
    - Remove unknown keys.
    - If action=='answer', ignore 'q' if present (no error).
    - On fatal errors (e.g., missing/invalid 'action', or missing/empty 'q' for search),
      return ({}, errors).
    """
```


In [7]:
from typing import Any, Dict, List, Tuple


In [8]:
ALLOWED_ACTIONS: set[str] = {"search", "answer"}
DEFAULT_K = 3
MIN_K = 1
MAX_K = 5

In [9]:
def _coerce_int(value: Any) -> Any:
    if isinstance(value, int):
        return value

    if isinstance(value, bool):
        return None
        
    if isinstance(value, str):
        stripped = value.strip()
        try:
            return int(stripped)
        except ValueError:
            return None
    return None


def _validate_action(action: Any, errors: List[str]) -> Any:
    if not isinstance(action, str):
        errors.append("action must be one of 'search' or 'answer'.")
        return None

    stripped_lowered_action = action.strip().lower()

    if stripped_lowered_action not in ALLOWED_ACTIONS:
        errors.append("action must be one of 'search' or 'answer'.")
        return None
        
    return stripped_lowered_action


def _validate_query(query: Any, errors: List[str]) -> Any:
    if query is None:
        errors.append("q is required when action is 'search'.")
        return None
        
    if not isinstance(query, str):
        errors.append("q must be a string when action is 'search'.")
        return None
    
    stripped_query = query.strip()
    if not stripped_query:
        errors.append("q must be a non-empty string when action is 'search'.")
        return None
        
    return stripped_query


def _validate_k(k: Any, errors: List[str]) -> int:
    if k is None:
        return DEFAULT_K

    coerced = _coerce_int(k)
    if coerced is None:
        errors.append("k must be an integer between 1 and 5, defaulting to 3.")
        return DEFAULT_K
    if not (MIN_K <= coerced <= MAX_K):
        errors.append("k must be between 1 and 5, defaulting to 3.")
        return DEFAULT_K
    return coerced


In [10]:
def validate_tool_call(payload: Dict[str, Any]) -> Tuple[Dict[str, Any], List[str]]:
    errors: List[str] = []
    clean: Dict[str, Any] = {}

    action = payload.get("action")
    query = payload.get("q")
    k = payload.get("k")

    action_value = _validate_action(action, errors)
    
    if action_value is None:
        return {}, errors

    if action_value == "search":
        query_value = _validate_query(query, errors)
        if query_value is None:
            return {}, errors
        clean["q"] = query_value

    k_value = _validate_k(k, errors)

    clean["action"] = action_value
    clean["k"] = k_value

    return clean, errors


In [ ]:
def test_validate_tool_call():    
    # Test 1: Valid search with all fields
    result, errors = validate_tool_call({"action": "search", "q": "test query", "k": 5})
    assert result == {"action": "search", "q": "test query", "k": 5}
    assert errors == []
    print("✓ Test 1 passed: Valid search with all fields")
    
    # Test 2: Valid search with default k
    result, errors = validate_tool_call({"action": "search", "q": "test"})
    assert result == {"action": "search", "q": "test", "k": 3}
    assert errors == []
    print("✓ Test 2 passed: Valid search with default k")
    
    # Test 3: Valid answer action
    result, errors = validate_tool_call({"action": "answer"})
    assert result == {"action": "answer", "k": 3}
    assert "q" not in result
    assert errors == []
    print("✓ Test 3 passed: Valid answer action")
    
    # Test 4: Answer with q present (should ignore q, no error)
    result, errors = validate_tool_call({"action": "answer", "q": "ignored"})
    assert result == {"action": "answer", "k": 3}
    assert "q" not in result
    assert errors == []
    print("✓ Test 4 passed: Answer with q present (ignored)")
    
    # Test 5: Action with whitespace and uppercase (should normalize)
    result, errors = validate_tool_call({"action": "  SEARCH  ", "q": "test", "k": 2})
    assert result == {"action": "search", "q": "test", "k": 2}
    assert errors == []
    print("✓ Test 5 passed: Action normalization (whitespace/uppercase)")
    
    # Test 6: Query with whitespace (should trim)
    result, errors = validate_tool_call({"action": "search", "q": "  trimmed query  ", "k": 1})
    assert result == {"action": "search", "q": "trimmed query", "k": 1}
    assert errors == []
    print("✓ Test 6 passed: Query trimming")
    
    # Test 7: k as string (should coerce)
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": "4"})
    assert result == {"action": "search", "q": "test", "k": 4}
    assert errors == []
    print("✓ Test 7 passed: k coercion from string")
    
    # Test 8: k as string with whitespace (should coerce)
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": "  2  "})
    assert result == {"action": "search", "q": "test", "k": 2}
    assert errors == []
    print("✓ Test 8 passed: k coercion from string with whitespace")
    
    # Test 9: Unknown keys should be removed
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": 3, "unknown": "value", "extra": 123})
    assert result == {"action": "search", "q": "test", "k": 3}
    assert "unknown" not in result
    assert "extra" not in result
    assert errors == []
    print("✓ Test 9 passed: Unknown keys removed")
    
    # Test 10: Missing action (fatal error)
    result, errors = validate_tool_call({"q": "test", "k": 3})
    assert result == {}
    assert len(errors) > 0
    assert any("action" in err.lower() for err in errors)
    print("✓ Test 10 passed: Missing action (fatal error)")
    
    # Test 11: Invalid action type (fatal error)
    result, errors = validate_tool_call({"action": 123, "q": "test"})
    assert result == {}
    assert len(errors) > 0
    print("✓ Test 11 passed: Invalid action type")
    
    # Test 12: Invalid action value (fatal error)
    result, errors = validate_tool_call({"action": "invalid", "q": "test"})
    assert result == {}
    assert len(errors) > 0
    print("✓ Test 12 passed: Invalid action value")
    
    # Test 13: Missing q for search (fatal error)
    result, errors = validate_tool_call({"action": "search", "k": 3})
    assert result == {}
    assert len(errors) > 0
    assert any("q" in err.lower() or "required" in err.lower() for err in errors)
    print("✓ Test 13 passed: Missing q for search (fatal error)")
    
    # Test 14: Empty q for search (fatal error)
    result, errors = validate_tool_call({"action": "search", "q": "", "k": 3})
    assert result == {}
    assert len(errors) > 0
    print("✓ Test 14 passed: Empty q for search (fatal error)")
    
    # Test 15: Whitespace-only q for search (fatal error)
    result, errors = validate_tool_call({"action": "search", "q": "   ", "k": 3})
    assert result == {}
    assert len(errors) > 0
    print("✓ Test 15 passed: Whitespace-only q for search (fatal error)")
    
    # Test 16: Non-string q for search (fatal error)
    result, errors = validate_tool_call({"action": "search", "q": 123, "k": 3})
    assert result == {}
    assert len(errors) > 0
    print("✓ Test 16 passed: Non-string q for search (fatal error)")
    
    # Test 17: k out of range (too low) - should default with error
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": 0})
    assert result == {"action": "search", "q": "test", "k": 3}
    assert len(errors) > 0
    assert any("between 1 and 5" in err.lower() for err in errors)
    print("✓ Test 17 passed: k out of range (too low)")
    
    # Test 18: k out of range (too high) - should default with error
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": 6})
    assert result == {"action": "search", "q": "test", "k": 3}
    assert len(errors) > 0
    print("✓ Test 18 passed: k out of range (too high)")
    
    # Test 19: k as bool (True becomes 1 because bool is subclass of int)
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": True})
    assert result == {"action": "search", "q": "test", "k": 1}  # True == 1 in Python
    assert errors == []  # No error because 1 is valid
    print("✓ Test 19 passed: k as bool True (coerced to 1)")
    
    # Test 19b: k as bool False (becomes 0, out of range, should default)
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": False})
    assert result == {"action": "search", "q": "test", "k": 3}  # False == 0, out of range
    assert len(errors) > 0
    print("✓ Test 19b passed: k as bool False (coerced to 0, defaults)")
    
    # Test 20: k as non-numeric string (should default with error)
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": "abc"})
    assert result == {"action": "search", "q": "test", "k": 3}
    assert len(errors) > 0
    print("✓ Test 20 passed: k as non-numeric string")
    
    # Test 21: k boundary values (1 and 5 should be valid)
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": 1})
    assert result == {"action": "search", "q": "test", "k": 1}
    assert errors == []
    print("✓ Test 21 passed: k boundary value (1)")
    
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": 5})
    assert result == {"action": "search", "q": "test", "k": 5}
    assert errors == []
    print("✓ Test 22 passed: k boundary value (5)")
    
    # Test 23: Empty payload
    result, errors = validate_tool_call({})
    assert result == {}
    assert len(errors) > 0
    print("✓ Test 23 passed: Empty payload")
    
    # Test 24: k as None (should default, no error)
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": None})
    assert result == {"action": "search", "q": "test", "k": 3}
    assert errors == []
    print("✓ Test 24 passed: k as None (defaults)")
    
    # Test 25: k as float string (should coerce if possible)
    result, errors = validate_tool_call({"action": "search", "q": "test", "k": "3.0"})
    # int("3.0") will fail, so should default
    assert result == {"action": "search", "q": "test", "k": 3}
    assert len(errors) > 0  # Should have error since "3.0" can't be converted to int directly
    print("✓ Test 25 passed: k as float string")
    
    print("\n🎉 All tests passed!")

# Run the tests
test_validate_tool_call()


✓ Test 1 passed: Valid search with all fields
✓ Test 2 passed: Valid search with default k
✓ Test 3 passed: Valid answer action
✓ Test 4 passed: Answer with q present (ignored)
✓ Test 5 passed: Action normalization (whitespace/uppercase)
✓ Test 6 passed: Query trimming
✓ Test 7 passed: k coercion from string
✓ Test 8 passed: k coercion from string with whitespace
✓ Test 9 passed: Unknown keys removed
✓ Test 10 passed: Missing action (fatal error)
✓ Test 11 passed: Invalid action type
✓ Test 12 passed: Invalid action value
✓ Test 13 passed: Missing q for search (fatal error)
✓ Test 14 passed: Empty q for search (fatal error)
✓ Test 15 passed: Whitespace-only q for search (fatal error)
✓ Test 16 passed: Non-string q for search (fatal error)
✓ Test 17 passed: k out of range (too low)
✓ Test 18 passed: k out of range (too high)
✓ Test 19 passed: k as bool True (coerced to 1)
✓ Test 19b passed: k as bool False (coerced to 0, defaults)
✓ Test 20 passed: k as non-numeric string
✓ Test 21 pass